In [45]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.utils import to_categorical

# 1. Load Data
def load_data(path):
    data = []  # To store sequences (shape: [num_samples, sequence_length, num_features])
    labels = []  # To store corresponding labels

    # Find the maximum sequence length
    max_sequence_length = 0
    for file_name in os.listdir(path):
        if file_name.endswith(".csv"):
            # Load the CSV file
            file_path = os.path.join(path, file_name)
            df = pd.read_csv(file_path)
            max_sequence_length = max(max_sequence_length, len(df))

    # Iterate through files in the folder
    for file_name in os.listdir(path):
        if file_name.endswith(".csv"):
            # Extract label from the file name (before the first '_')
            label = file_name.split("_")[0]

            # Load the CSV file
            file_path = os.path.join(path, file_name)
            df = pd.read_csv(file_path)
            sequence_data = df.values  # Convert to NumPy array

            # Add padding to match the maximum sequence length
            padded_sequence = np.zeros((max_sequence_length, sequence_data.shape[1]))
            padded_sequence[:len(sequence_data), :] = sequence_data  # Fill with actual data
            data.append(padded_sequence)

            # Add the label for this sequence
            labels.append(label)

    return np.array(data), np.array(labels)

# Path to the gesture data folder
path = "gesture_data"
data, labels = load_data(path)

# 2. Preprocess Data
# Encode labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)  # Convert to one-hot encoding

# 3. Define Model
model = Sequential([
    # Masking layer to ignore padded zeros
    Masking(mask_value=0.0, input_shape=(data.shape[1], data.shape[2])),
    # LSTM layer for sequence data
    LSTM(64, return_sequences=False),
    # Fully connected layer
    Dense(32, activation='relu'),
    # Output layer
    Dense(categorical_labels.shape[1], activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 4. Train Model
history = model.fit(
    data, categorical_labels,
    validation_split=0.2,
    epochs=20,
    batch_size=16
)

# Save the model
model.save("gesture_model.h5")

# 6. Decode Predictions (Example)
def predict_sequence(sequence):
    sequence = np.expand_dims(sequence, axis=0)  # Add batch dimension
    prediction = model.predict(sequence)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

Epoch 1/20


/home/gins/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gins/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 6), output.shape=(None, 1)

In [44]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TensorFlow Lite ke file .tflite
with open("gesture_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpn60e_yk2/assets


INFO:tensorflow:Assets written to: /tmp/tmpn60e_yk2/assets


Saved artifact at '/tmp/tmpn60e_yk2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 16, 63), dtype=tf.float32, name='keras_tensor_39')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  132015436630144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132016410705440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132016410702272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132015436639472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132015436637008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132016696187312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132015436632784: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1737690467.454754   28048 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1737690467.454766   28048 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-01-24 10:47:47.454929: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpn60e_yk2
2025-01-24 10:47:47.455543: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-24 10:47:47.455566: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpn60e_yk2
2025-01-24 10:47:47.460822: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-24 10:47:47.486883: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpn60e_yk2
2025-01-24 10:47:47.498665: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 43737 microseconds.
loc(callsite(callsite(fused["CudnnRNNV3:", "s

ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_8_1/lstm_8_1/CudnnRNNV3@__inference_function_31174"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_31213"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.CudnnRNNV3' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_8_1/lstm_8_1/CudnnRNNV3@__inference_function_31174"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_31213"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): Error code: ERROR_NEEDS_CUSTOM_OPS
<unknown>:0: error: failed while converting: 'main': 
Some ops in the model are custom ops, See instructions to implement custom ops: https://www.tensorflow.or